# Prepare ini file for Sashimi plots for Differential splicing

In [2]:
import os
import gzip
import numpy as np
import pysam as ps
import pandas as pd
import pyBigWig as pw
import glob

ERROR: Error in parse(text = x, srcfile = src): <text>:1:8: unexpected symbol
1: import os
           ^


## Input data

Need these files: 
1. bigwig files: for each sample of two groups, average by group
2. PSI files: probably can use the ds output files

In [2]:
os.chdir('/home/chaodai/cdai/SpliFi/analysis')

In [3]:
!pwd

/project2/yangili1/cdai/SpliFi/analysis


### Bigwig files

In [4]:
!ls ../code/resources/GTEx/BigWig/Brain-Cortex/*.bw | head -3
!ls ../code/resources/GTEx/BigWig/Muscle-Skeletal/*.bw | head -3


!ls ../code/resources/GTEx/BigWig/Brain-Cortex/*.bw | wc -l
!ls ../code/resources/GTEx/BigWig/Muscle-Skeletal/*.bw | wc -l

../code/resources/GTEx/BigWig/Brain-Cortex/GTEX-1117F-3226-SM-5N9CT.bw
../code/resources/GTEx/BigWig/Brain-Cortex/GTEX-111FC-3126-SM-5GZZ2.bw
../code/resources/GTEx/BigWig/Brain-Cortex/GTEX-1128S-2726-SM-5H12C.bw
ls: write error: Broken pipe
../code/resources/GTEx/BigWig/Muscle-Skeletal/GTEX-1117F-0426-SM-5EGHI.bw
../code/resources/GTEx/BigWig/Muscle-Skeletal/GTEX-111CU-2026-SM-5GZZC.bw
../code/resources/GTEx/BigWig/Muscle-Skeletal/GTEX-111FC-0326-SM-5GZZ1.bw
ls: write error: Broken pipe
112
112


### Samples names used in DS analysis:

In [31]:

!head -3 ../code/results/ds/GTEx/Brain-Cortex_v_Muscle-Skeletal/ds_sample_group.txt
!tail -3 ../code/results/ds/GTEx/Brain-Cortex_v_Muscle-Skeletal/ds_sample_group.txt

Muscle-Skeletal.GTEX-1117F.tsv.gz Muscle-Skeletal
Muscle-Skeletal.GTEX-111CU.tsv.gz Muscle-Skeletal
Muscle-Skeletal.GTEX-111FC.tsv.gz Muscle-Skeletal


Brain-Cortex.GTEX-14BMV.tsv.gz Brain-Cortex
Brain-Cortex.GTEX-14C39.tsv.gz Brain-Cortex
Brain-Cortex.GTEX-14C5O.tsv.gz Brain-Cortex

### Intron junction PSI file

In [27]:
!head -3 ../code/results/ds/GTEx/Brain-Cortex_v_Muscle-Skeletal/ds_cluster_significance.txt

status	loglr	df	p	cluster	p.adjust
<=1 junction in cluster	NA	NA	NA	chr1:clu_10_+	NA
<=1 junction in cluster	NA	NA	NA	chr1:clu_1000_+	NA


In [26]:
!head -3 ../code/results/ds/GTEx/Brain-Cortex_v_Muscle-Skeletal/ds_effect_sizes.txt

intron	logef	Muscle-Skeletal	Brain-Cortex	deltapsi
chr1:75724817:75728388:clu_1002_+	-0.565955248841669	0.0025053453582882	0.00234306296179661	-0.000162282396491587
chr1:75724817:75728400:clu_1002_+	-0.578350460709226	0.445147405737144	0.411184781360074	-0.0339626243770698


### Intron counts file

used to get productive vs. unproductive introns

In [51]:
!ls ../code/results/ds/GTEx/Brain-Cortex_v_Muscle-Skeletal/ds_perind_numers.counts.noise_by_intron.gz
!zcat ../code/results/ds/GTEx/Brain-Cortex_v_Muscle-Skeletal/ds_perind_numers.counts.noise_by_intron.gz | head -3 | cut -d " " -f 1-5

../code/results/ds/GTEx/Brain-Cortex_v_Muscle-Skeletal/ds_perind_numers.counts.noise_by_intron.gz


chrom Muscle-Skeletal.GTEX-1117F.tsv.gz Muscle-Skeletal.GTEX-111CU.tsv.gz Muscle-Skeletal.GTEX-111FC.tsv.gz Muscle-Skeletal.GTEX-111VG.tsv.gz
chr1:779092:781936:clu_3_+:IN 0 0 0 0
chr1:779092:803918:clu_3_+:IN 0 0 0 0

gzip: stdout: Broken pipe


In [89]:
# read in only the first 2 columns
introns = pd.read_csv('../code/results/ds/GTEx/Brain-Cortex_v_Muscle-Skeletal/ds_perind_numers.counts.noise_by_intron.gz', sep=' ', usecols=[0])

In [90]:
introns = [(':'.join(x[:-1]), f'{x[0]}:{x[3]}', x[4]) for x in introns.chrom.str.split(':')]
introns = {x[0]: x[2] for x in introns}

TypeError: 'StringMethods' object is not iterable

In [6]:
# get available bigwig files in each tissue

tissue1 = 'Brain-Cortex'
tissue2 = 'Muscle-Skeletal'

In [7]:
# Brain-Cortex bigwig files
bw_files1 = glob.glob(f'../code/resources/GTEx/BigWig/{tissue1}/*.bw')
# Muscle bigwig files
bw_files2 = glob.glob(f'../code/resources/GTEx/BigWig/{tissue2}/*.bw')

In [8]:
# Extract sample names from bigwig file names, the second and third group after splitting by '-'
bw_samples1 = [(f'{xx[0]}-{xx[1]}') for xx in [x.split('/')[-1].split('-') for x in bw_files1]]
bw_samples2 = [(f'{xx[0]}-{xx[1]}') for xx in [x.split('/')[-1].split('-') for x in bw_files2]]

In [9]:
# make a dictionary with bw_samples as keys and bw_files as values
bw_dict1 = dict(zip(bw_samples1, bw_files1))
bw_dict2 = dict(zip(bw_samples2, bw_files2))

In [10]:
# samples from ds sample groups
ds_samples = pd.read_csv(f'../code/results/ds/GTEx/{tissue1}_v_{tissue2}/ds_sample_group.txt',
                         sep=' ', header=None, names=['fname', 'tissue'])

In [11]:
ds_samples['sname'] = [x.split('.')[1] for x in ds_samples['fname']]

In [12]:
ds_samples1 = ds_samples[ds_samples['tissue'] == tissue1].sname.to_list()
ds_samples2 = ds_samples[ds_samples['tissue'] == tissue2].sname.to_list()

In [13]:
# plots are based on the intersect of bw samples and ds samples
pl_samples1 = list(set(bw_samples1) & set(ds_samples1))
pl_samples2 = list(set(bw_samples2) & set(ds_samples2))

In [14]:
# plots use these bw files
pl_bw_files1 = [bw_dict1[x] for x in pl_samples1]
pl_bw_files2 = [bw_dict2[x] for x in pl_samples2]

In [15]:
len(pl_samples1), len(pl_samples2)
len(ds_samples1), len(ds_samples2)
len(bw_files1),  len(bw_files2)

(78, 78)

(80, 80)

(112, 112)

In [16]:
def avgBigwig(bwfiles, grange):
    '''Average multiple bigwig files in a specific region
    
    bwfiles : list of bigwig files (path).
    grange  : tuple. Genomic range, BED like 0-based coordinates, 
              eg. ('chr1', 25101, 27101)
    
    return  : a dictionary of keys: 
        - header, for pyBigWig to write as header
        - values, for pyBigwig to addentries as values
    '''
    chrom, start, end = grange
    values = []
    bwo = {}
    for bw in bwfiles:
        if not os.path.isfile(bw): continue
        with pw.open(bw, 'rt') as b:
            header = list(b.chroms().items())
            vals = b.values(chrom, start, end, numpy=True)
            vals = np.nan_to_num(vals)
            values.append(vals)
    
    if values != [] and header != []:
        avgValues = np.mean(values, axis=0)
        bwo = {'header': header, 'values': avgValues}
    return bwo



In [17]:
20685020-20685251

-231

In [18]:
extend = 10000
grange = 'chr1:15651895:15656616:clu_261_+'
c, s, e, clu = grange.split(':')
c,s,e = c, int(s) - extend, int(e) + extend

In [19]:
c, s, e
clu

('chr1', 15641895, 15666616)

'clu_261_+'

In [20]:
avgBW1 = avgBigwig(pl_bw_files1, (c,s,e))
avgBW2 = avgBigwig(pl_bw_files2, (c,s,e))

In [22]:
with pw.open(f'test-{tissue1}.bw', 'w') as b:
  b.addHeader(avgBW1['header'])
  b.addEntries(c, s, values=avgBW1['values'], span=1, step=1)

In [25]:
with pw.open(f'test-{tissue2}.bw', 'w') as b:
  b.addHeader(avgBW2['header'])
  b.addEntries(c, s, values=avgBW2['values'], span=1, step=1)

In [26]:
!ls -lah *bw

-rw-rw-r-- 1 chaodai pi-yangili1 35K Apr 11 10:58 test-Brain-Cortex.bw
-rw-rw-r-- 1 chaodai pi-yangili1 44K Apr 11 10:58 test-Muscle-Skeletal.bw


In [92]:
# get splicing links
psi = pd.read_csv(f'../code/results/ds/GTEx/{tissue1}_v_{tissue2}/ds_effect_sizes.txt', sep= '\t')
psi['itype'] = [introns.get(x) for x in psi['intron']]

In [93]:
psi[psi.intron.str.contains(clu)]

,intron,logef,Muscle-Skeletal,Brain-Cortex,deltapsi,itype
2875,chr1:15651895:15656420:clu_261_+,-2.090066,0.513989,0.01592,-0.498069,UP
2876,chr1:15651895:15656616:clu_261_+,2.090066,0.486011,0.98408,0.498069,PR


In [94]:
lnk1 = psi[psi.intron.str.contains(clu)][['intron', tissue1, 'itype']]
lnk2 = psi[psi.intron.str.contains(clu)][['intron', tissue2, 'itype']]

In [95]:
lnk1
lnk2

,intron,Brain-Cortex,itype
2875,chr1:15651895:15656420:clu_261_+,0.01592,UP
2876,chr1:15651895:15656616:clu_261_+,0.98408,PR


,intron,Muscle-Skeletal,itype
2875,chr1:15651895:15656420:clu_261_+,0.513989,UP
2876,chr1:15651895:15656616:clu_261_+,0.486011,PR


In [100]:
lnk1['chrom1'] = lnk1.intron.str.split(':').str[0]
lnk1['start1'] = lnk1.intron.str.split(':').str[1]
lnk1['start2'] = lnk1.start1
lnk1['chrom2'] = lnk1.chrom1
lnk1['end1'] = lnk1.intron.str.split(':').str[2]
lnk1['end2'] = lnk1.end1

In [101]:
lnk2['chrom1'] = lnk2.intron.str.split(':').str[0]
lnk2['start1'] = lnk2.intron.str.split(':').str[1]
lnk2['start2'] = lnk2.start1
lnk2['chrom2'] = lnk2.chrom1
lnk2['end1'] = lnk2.intron.str.split(':').str[2]
lnk2['end2'] = lnk2.end1

In [102]:
lnk1

,intron,Brain-Cortex,itype,chrom1,start1,start2,chrom2,end1,end2
2875,chr1:15651895:15656420:clu_261_+,0.01592,UP,chr1,15651895,15651895,chr1,15656420,15656420
2876,chr1:15651895:15656616:clu_261_+,0.98408,PR,chr1,15651895,15651895,chr1,15656616,15656616


In [103]:
lnk1_o = lnk1[['chrom1', 'start1', 'start2', 'chrom2', 'end1', 'end2', tissue1]]
lnk2_o = lnk2[['chrom1', 'start1', 'start2', 'chrom2', 'end1', 'end2', tissue2]]

In [206]:
lnk1_o
lnk2_o

,chrom1,start1,start2,chrom2,end1,end2,Brain-Cortex
2875,chr1,15651895,15651895,chr1,15656420,15656420,0.01592
2876,chr1,15651895,15651895,chr1,15656616,15656616,0.98408


,chrom1,start1,start2,chrom2,end1,end2,Muscle-Skeletal
2875,chr1,15651895,15651895,chr1,15656420,15656420,0.513989
2876,chr1,15651895,15651895,chr1,15656616,15656616,0.486011


In [39]:
lnk1_o.to_csv(f'test-lnk-{tissue1}.tsv', sep='\t', index=False, header=False)
lnk2_o.to_csv(f'test-lnk-{tissue2}.tsv', sep='\t', index=False, header=False)

In [76]:
lnk1

,intron,Brain-Cortex,chrom1,start1,start2,chrom2,end1,end2
2875,chr1:15651895:15656420:clu_261_+,0.01592,chr1,15651895,15651895,chr1,15656420,15656420
2876,chr1:15651895:15656616:clu_261_+,0.98408,chr1,15651895,15651895,chr1,15656616,15656616


In [104]:
# concatenate lnk1 and lnk2
pd.concat([
  lnk1[lnk1.itype == 'UP'][['chrom1', 'start1', 'start2', 'chrom2', 'end1', 'end2']],
  lnk2[lnk2.itype == 'UP'][['chrom1', 'start1', 'start2', 'chrom2', 'end1', 'end2']]
  ], axis=0).drop_duplicates().to_csv(f'test-lnk-UP.tsv', sep='\t', index=False, header=False)

In [107]:
pd.concat([
  lnk1[lnk1.itype == 'PR'][['chrom1', 'start1', 'start2', 'chrom2', 'end1', 'end2']],
  lnk2[lnk2.itype == 'PR'][['chrom1', 'start1', 'start2', 'chrom2', 'end1', 'end2']]
  ], axis=0).drop_duplicates().to_csv(f'test-lnk-PR.tsv', sep='\t', index=False, header=False)

### check: are my ds samples the same as my dge samples?